In [ ]:
# %load ../../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC, NWTG
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
import zcode.math as zmath

In [ ]:
import holodeck.simple_sam

In [ ]:
fobs_yr = 1.0 / YR

In [ ]:
sam_simple = holo.simple_sam.Simple_SAM()
gwb_simple = sam_simple.gwb_ideal(fobs_yr)
print(gwb_simple)

In [ ]:
# gsmf = holo.sam.GSMF_Schechter()
# gpf = holo.sam.GPF_Power_Law()
# gmt = holo.sam.GMT_Power_Law()
mmbulge = holo.relations.MMBulge_Standard(
    mamp=sam_simple._mbh_star, mplaw=sam_simple._alpha_mbh_star, mref=1e11*MSOL
)
# sam = holo.sam.Semi_Analytic_Model(gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge, shape=100)
sam = holo.sam.Semi_Analytic_Model(mmbulge=mmbulge)
gwb = sam.gwb_ideal(fobs_yr)
print(gwb)

In [ ]:
gwb, gwb_simple, gwb/gwb_simple

In [ ]:
NUM = 100
mgal = MSOL * (10.0 ** np.random.uniform(8, 12, NUM))
qgal = 10.0 ** np.random.uniform(-3.0, 0.0, NUM)
redz = zmath.random_power([0.01, 1.5], 2.0, NUM)

In [ ]:
def check(mm, qq, zz, check, simple):
    err = (check - simple) / np.min([check, simple], axis=0)
    isclose = np.isclose(check, simple, rtol=1e-2, atol=0.0)

    print(zmath.str_array(check))
    print(zmath.str_array(simple))
    print(zmath.stats_str(check))
    print(zmath.stats_str(simple))
    print(zmath.str_array(err))
    print(zmath.stats_str(err))
    print(isclose)

    fig, axes = plot.figax(ncols=3)
    ax = axes[0]
    xx = mm
    ax.scatter(xx, check, color='r', alpha=0.5, marker='+')
    ax.scatter(xx, simple, color='b', alpha=0.5, marker='x')
    tw = ax.twinx()
    ii = np.argsort(xx)
    tw.plot(xx[ii], err[ii], alpha=0.25)

    ax = axes[1]
    xx = qq
    ax.scatter(xx, check, color='r', alpha=0.5, marker='+')
    ax.scatter(xx, simple, color='b', alpha=0.5, marker='x')
    tw = ax.twinx()
    ii = np.argsort(xx)
    tw.plot(xx[ii], err[ii], alpha=0.25)

    ax = axes[2]
    xx = zz
    ax.scatter(xx, check, color='r', alpha=0.5, marker='+')
    ax.scatter(xx, simple, color='b', alpha=0.5, marker='x')
    tw = ax.twinx()
    ii = np.argsort(xx)
    tw.plot(xx[ii], err[ii], alpha=0.25)

    return isclose
    
def frac_diff(v1, v2):
    ee = (v2 - v1)/np.min([v1, v2], axis=0)
    return ee

def frac_truth(yy, truth):
    yy = yy if (truth is None) else np.fabs(yy - truth) / truth
    return yy

# Check Components

## GSMF

In [ ]:
gsmf_check = sam._gsmf(mgal, redz)
gsmf_simple = sam_simple.gsmf(mgal, redz)
err = (gsmf_check - gsmf_simple) / np.min([gsmf_check, gsmf_simple], axis=0)
print(np.isclose(gsmf_check, gsmf_simple, rtol=1e-2))
# print(err)

fig, axes = plot.figax(ncols=2)
ax = axes[0]
ax.scatter(mgal, gsmf_check, color='r', alpha=0.5, marker='+')
ax.scatter(mgal, gsmf_simple, color='b', alpha=0.5, marker='x')

ax = axes[1]
ax.scatter(qgal, gsmf_check, color='r', alpha=0.5, marker='+')
ax.scatter(qgal, gsmf_simple, color='b', alpha=0.5, marker='x')
plt.show()

## GPF

In [ ]:
# gsmf = holo.sam.GSMF_Schechter()
# gpf = holo.sam.GPF_Power_Law()
# gmt = holo.sam.GMT_Power_Law()
# sam = holo.sam.Semi_Analytic_Model(gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge, shape=100)
gpf_alpha = 0.0
gpf = holo.sam.GPF_Power_Law(malpha=gpf_alpha)
sam = holo.sam.Semi_Analytic_Model(mmbulge=mmbulge, gpf=gpf)
sam_simple = holo.simple_sam.Simple_SAM(gpf_alpha=gpf_alpha)

gpf_check = sam._gpf(mgal, qgal, redz)
gpf_simple = sam_simple.gpf(mgal, qgal, redz)

check(mgal, qgal, redz, gpf_check, gpf_simple)
plt.show()

## GMT

In [ ]:
gmt_check = sam._gmt(mgal, qgal, redz)
gmt_simple = sam_simple.gmt(mgal, qgal, redz)
check(mgal, qgal, redz, gmt_check, gmt_simple)
plt.show()

## Galaxy NDens

In [ ]:
sam_simple = holo.simple_sam.Simple_SAM()
n1 = sam_simple.ndens_galaxy(mgal, qgal, redz)
n2 = sam_simple._ndens_galaxy_check(mgal, qgal, redz)

check(mgal, qgal, redz, n1, n2)
plt.show()

In [ ]:
sam = holo.sam.Semi_Analytic_Model()
ndg_check = sam._ndens_gal(mgal, qgal, redz)
ndg_simple = sam_simple.ndens_galaxy(mgal, qgal, redz)

uu_check = ndg_check; vv_simple = ndg_simple
check(mgal, qgal, redz, uu_check, vv_simple)
plt.show()

## MBH NDens

In [ ]:
sam_simple = holo.simple_sam.Simple_SAM()
sam = holo.sam.Semi_Analytic_Model(mmbulge=mmbulge)

uu = sam._ndens_mbh(mgal, qgal, redz)
vv = sam_simple.ndens_mbh(mgal, qgal, redz)

close = check(mgal, qgal, redz, uu, vv)
if not all(close):
    err_msg = "MBH Number-Density mismatch between SAM and Simple_SAM!"
    raise ValueError(err_msg)

plt.show()

In [ ]:
sam._density = None
ndens_sam = sam.static_binary_density
mbh_tot, mbh_rat, redz = np.copy(sam.grid)
mbh_pri, mbh_sec = utils.m1m2_from_mtmr(mbh_tot, mbh_rat)
mst_pri, mst_sec = [mmbulge.mstar_from_mbh(_mbh, scatter=False) for _mbh in [mbh_pri, mbh_sec]]
mst_rat = mst_sec/mst_pri

ndens_ref = sam._ndens_mbh(mst_pri, mst_rat, redz)

err = frac_diff(ndens_sam, ndens_ref)
print(zmath.stats_str(ndens_sam))
print(zmath.stats_str(ndens_ref))
print("err=", zmath.stats_str(err))

if not np.allclose(ndens_sam, ndens_ref, rtol=1e-6, atol=0.0):
    err_msg = f"sam mbh ndens does not match consistency check || error too large!"
    raise ValueError(err_msg)


## GWB Ideal

In [ ]:
sam_simple = holo.simple_sam.Simple_SAM(size=100)
sam = holo.sam.Semi_Analytic_Model(mmbulge=mmbulge)

hc1 = sam_simple.gwb_sam(fobs_yr, sam, dlog10=False)
hc2 = sam_simple.gwb_sam(fobs_yr, sam, dlog10=True)
print(hc1, hc2, hc1/hc2)
if not np.isclose(hc1, hc2, rtol=1e-2, atol=0.0):
    err = f"{hc1=:.8e} (gwb) vs. {hc2=:.8e} || error too large!"
    raise ValueError(err)

plt.show()

In [ ]:
sam_simple = holo.simple_sam.Simple_SAM()

mg = sam_simple.mass_gal[:, np.newaxis, np.newaxis]
qg = sam_simple.mrat_gal[np.newaxis, :, np.newaxis]
rz = sam_simple.redz[np.newaxis, np.newaxis, :]
mtot = sam_simple.mbh[:, :, np.newaxis]

ndens_a1 = sam_simple.ndens_mbh(mg, qg, rz, dlog10=False)
ndens_b1 = ndens_a1 * mtot * np.log(10.0)
ndens_b2 = sam_simple.ndens_mbh(mg, qg, rz, dlog10=True)
ndens_a2 = ndens_b2 / (mtot * np.log(10.0))

print("dlog10(M)")
print(utils.stats(ndens_b1, prec=4))
print(utils.stats(ndens_b2, prec=4))
err = frac_diff(ndens_b1, ndens_b2)
print(utils.stats(err, prec=2))
assert np.allclose(ndens_b1, ndens_b2)

print("dM")
print(utils.stats(ndens_a1, prec=4))
print(utils.stats(ndens_a2, prec=4))
err = frac_diff(ndens_a1, ndens_a2)
print(utils.stats(err[err > 0], prec=2))
assert np.allclose(ndens_a1, ndens_a2)

In [ ]:
sam_simple = holo.simple_sam.Simple_SAM()

hc1 = sam_simple.gwb_ideal(fobs_yr, dlog10=True)
hc2 = sam_simple.gwb_ideal(fobs_yr, dlog10=False)
print(hc1, hc2, hc1/hc2)
if not np.isclose(hc1, hc2, rtol=1e-2, atol=0.0):
    err = f"{hc1=:.8e} (gwb) vs. {hc2=:.8e} || error too large!"
    raise ValueError(err)

# check(mgal, qgal, redz, uu, vv)
plt.show()

In [ ]:
sam_simple = holo.simple_sam.Simple_SAM()
sam = holo.sam.Semi_Analytic_Model(mmbulge=mmbulge)

hc1 = sam_simple.gwb_ideal(fobs_yr)
hc2 = sam_simple.gwb_sam(fobs_yr, sam)
print(hc1, hc2, hc1/hc2)
if not np.isclose(hc1, hc2, rtol=1e-5, atol=0.0):
    err = f"{hc1=:.8e} (gwb) vs. {hc2=:.8e} || error too large!"
    raise ValueError(err)

# check(mgal, qgal, redz, uu, vv)
plt.show()

In [ ]:
val = 0.0
# val = -0.8
gpf = holo.sam.GPF_Power_Law(malpha=val)

sam_simple = holo.simple_sam.Simple_SAM(gpf_alpha=val)
sam = holo.sam.Semi_Analytic_Model(mmbulge=mmbulge, gpf=gpf)

hc1 = sam.gwb_ideal(fobs_yr)
hc2 = sam_simple.gwb_ideal(fobs_yr)

err = frac_diff(hc1, hc2)

print(hc1, hc2, err)
if not np.isclose(hc1, hc2, rtol=1e-2, atol=0.0):
    err_msg = f"{hc1=:.8e} (gwb) vs. {hc2=:.8e} || error too large!"
    raise ValueError(err_msg)


# Vary Parameters

In [ ]:
sam_full = holo.sam.Semi_Analytic_Model(mmbulge=mmbulge)
sam_simp = holo.simple_sam.Simple_SAM()
gwb_full = sam_full.gwb_ideal(fobs_yr)
gwb_simp = sam_simp.gwb_ideal(fobs_yr)
print("\t", gwb_full)
print("\t", gwb_simp)
err = frac_diff(gwb_full, gwb_simp)
print("\t", err)

assert np.allclose(gwb_full, gwb_simp, rtol=1e-2, atol=0.0)


Define all of the parameters that are going to be varied, and specify information about their range of values to randomly draw from, the classes they belong to in the SAM model, etc.

In [ ]:
params = {
    'gsmf_phi0_const': [
        [-2.77, -0.29, +0.27],
        ['gsmf', holo.sam.GSMF_Schechter],
        ['phi0', None]
    ],
    'gsmf_phiz': [
        [-0.27, -0.21, +0.23],
        ['gsmf', holo.sam.GSMF_Schechter],
        ['phiz', None]
    ],
    'gsmf_log10m0': [
        [+11.24, -0.17, +0.20],
        ['gsmf', holo.sam.GSMF_Schechter],
        ['mref0', lambda xx: np.power(10.0, xx) * MSOL],
    ],
    'gsmf_alpha0_const': [
        [-1.24, -0.16, +0.16],
        ['gsmf', holo.sam.GSMF_Schechter],
        ['alpha0', None]
    ],
    'gsmf_alphaz': [
        [-0.03, -0.14, +0.16],
        ['gsmf', holo.sam.GSMF_Schechter],
        ['alphaz', None]
    ],
    
    'gpf_norm': [
        [0.02, 0.03], 
        ['gpf', holo.sam.GPF_Power_Law],
        ['frac_norm_allq', None]
    ],
    'gpf_alpha': [
        [-0.2, +0.2], 
        ['gpf', holo.sam.GPF_Power_Law],
        ['malpha', None]
    ],
    'gpf_beta': [
        [0.6, 1.0], 
        ['gpf', holo.sam.GPF_Power_Law],
        ['zbeta', None]
    ],
    'gpf_gamma': [
        [-0.2, +0.2], 
        ['gpf', holo.sam.GPF_Power_Law],
        ['qgamma', None]
    ],

    'gmt_norm': [
        [0.1, 2.0], 
        ['gmt', holo.sam.GMT_Power_Law],
        ['time_norm', lambda xx: xx*GYR]
    ],
    'gmt_alpha': [
        [-0.2, +0.2], 
        ['gmt', holo.sam.GMT_Power_Law],
        ['malpha', None]
    ],
    'gmt_beta': [
        [-2, +1], 
        ['gmt', holo.sam.GMT_Power_Law],
        ['zbeta', None]
    ],
    'gmt_gamma': [
        [-0.2, +0.2], 
        ['gmt', holo.sam.GMT_Power_Law],
        ['qgamma', None]
    ],

    'mbh_star_log10': [
        [8.17, -0.32, +0.35], 
        ['mmbulge', holo.relations.MMBulge_Standard],
        ['mamp', lambda xx: np.power(10.0, xx) * MSOL]
    ],
    'alpha_mbh_star': [
        [1.01, -0.10, +0.08], 
        ['mmbulge', holo.relations.MMBulge_Standard],
        ['mplaw', None]
    ],
}

In [ ]:
# Choose some frequencies at which to compare
freqs = utils.nyquist_freqs(5*YR, 0.2*YR)

def get_param_from_bounds(bounds):
    # If there are only two bounds, it means to sample uniformly between them
    if len(bounds) == 2:
        xx = np.random.uniform(*bounds)
    # If there are 3 bounds, it means to sample from a normal distribution
    #    bounds[0] is the mean, bounds[1] is the negative stdev, bounds[2] is the positive stdev
    elif len(bounds) == 3:
        # if stdev are the same, draw from symmetric normal distribution
        if bounds[1] == bounds[2]:
            xx = np.random.normal(bounds[0], bounds[1])
        # if stdev are NOT the same, draw from lower and upper normal distributions, and choose
        # randomly between them
        else:
            # choose [-, +] deviations from zero
            yy = [np.sign(bb) * np.fabs(np.random.normal(0.0, np.fabs(bb)))
                  for bb in bounds[1:]]

            xx = bounds[0]
            # choose which of the [-, +] to use, and add that to the mean value
            if np.random.rand() < 0.5:
                xx -= np.fabs(yy[0])
            else:
                xx += np.fabs(yy[1])

    else:
        raise ValueError()

    return xx


def compare_sam_with_parameter(freqs, par, params, xx=None):

    #     'gsmf_log10m0': [
    #         [+11.24, -0.17, +0.20],
    #         ['gsmf', holo.sam.GSMF_Schechter],
    #         ['mref0', lambda xx: np.power(10.0, xx) * MSOL],
    #     ],

    vals = params[par]
    print("\n", par, vals)
    bounds, _par, _arg = vals
    par_class_name, par_class = _par
    arg_name, arg_func = _arg

    if xx is None:
        xx = get_param_from_bounds(bounds)

    # ---- Initialize ----
    
    # Creater the simple SAM.  `par` gives the name of the argument, and `xx` is its value
    simp_kwargs = {par: xx}
    print("\t", simp_kwargs)
    sam_simp = holo.simple_sam.Simple_SAM(**simp_kwargs)

    # For the full SAM, need to instantiate the class that this parameter belongs to
    # transform the value as needed (e.g. log10(x/Msol) ==> y gram)
    xx = arg_func(xx) if arg_func is not None else xx
    # We always need to use an `mmbulge` to match between the SAM models
    sam_kwargs = {'mmbulge': mmbulge}
    # If we're modifying an `mmbulge` parameter, start with default values and update them
    if par_class_name == 'mmbulge':
        class_kwargs = dict(mamp=sam_simple._mbh_star, mplaw=sam_simple._alpha_mbh_star)
    else:
        class_kwargs = {}
    class_kwargs[arg_name] = xx

    # construct the class containing the parameter
    print("\t", class_kwargs)
    par_inst = par_class(**class_kwargs)
    # add the class, or update `mmbulge` if we're modifying that
    sam_kwargs[par_class_name] = par_inst
    sam_full = holo.sam.Semi_Analytic_Model(**sam_kwargs)

    # ---- Test / Compare Models ----

    # - Galaxy Number Density
    print("galaxy number density")
    full = sam._ndens_gal(mgal, qgal, redz)
    simp = sam_simple.ndens_galaxy(mgal, qgal, redz)
    err = frac_diff(full, simp)
    print("\t", full[:3])
    print("\t", simp[:3])
    print("\t", utils.stats(err), err.shape)
    assert np.allclose(full, simp, rtol=1e-3, atol=0.0)

    # - MBH Number Density
    print("galaxy number density")
    full = sam._ndens_mbh(mgal, qgal, redz)
    simp = sam_simple.ndens_mbh(mgal, qgal, redz)
    # full = sam._ndens_mbh(*sam.grid)
    # simp = sam_simple.ndens_mbh(*sam.grid)
    err = frac_diff(full, simp)
    print("\t", full[:3])
    print("\t", simp[:3])
    print("\t", utils.stats(err), err.shape)
    assert np.allclose(full, simp, rtol=1e-3, atol=0.0)

    full = sam.static_binary_density
    mbh_tot, mbh_rat, rz = np.copy(sam.grid)
    mbh_pri, mbh_sec = utils.m1m2_from_mtmr(mbh_tot, mbh_rat)
    mst_pri, mst_sec = [mmbulge.mstar_from_mbh(_mbh, scatter=False) for _mbh in [mbh_pri, mbh_sec]]
    mst_rat = mst_sec/mst_pri

    simp = sam._ndens_mbh(mst_pri, mst_rat, rz)
    print("\t", full.flatten()[:3])
    print("\t", simp.flatten()[:3])
    print("\t", utils.stats(err), err.shape)
    assert np.allclose(full, simp, rtol=1e-3, atol=0.0)

    # - GWB
    print("GWB")
    gwb_full = sam_full.gwb_ideal(freqs)
    gwb_simp = sam_simp.gwb_ideal(freqs)
    err = frac_diff(gwb_full, gwb_simp)
    print("\t", gwb_full[:3])
    print("\t", gwb_simp[:3])
    print("\t", utils.stats(err), err.shape)
    assert np.allclose(gwb_full, gwb_simp, rtol=1e-1, atol=0.0)
    return

params_list = params.keys()
params_list = ['gpf_alpha']
pval = None
pval = -1.2
    
for par in params_list:
    compare_sam_with_parameter(freqs, par, params, xx=pval)